In [23]:
from database import SessionLocal
from models import Question
from datetime import datetime

# db = SessionLocal()
# for i in range(300):
#     q = Question(subject="테스트 데이터입니다:[%03d]" %i, content="내용무", create_date=datetime.now())
#     db.add(q)
# db.commit()

In [24]:
# secret key 생성
# import secrets
# secrets.token_hex(32)

In [36]:
from database import SessionLocal
from models import Question, Answer
from models import User
db = SessionLocal()


ImportError: cannot import name 'User' from 'models' (c:\projects\myapi\models.py)

In [32]:
db.query(Question).outerjoin(Answer).filter(
    Question.content.ilike('%파이썬%') | 
    Answer.content.ilike('%파이썬%')).distinct().count()


1

In [37]:
sub_query = db.query(Answer.question_id, Answer.content, User.username)\
    .outerjoin(User, Answer.user_id == User.id).subquery()

NameError: name 'User' is not defined

In [17]:
db.query(Answer.question_id, Answer.content).all()

[(2, '네 자동으로 생성됩니다.'),
 (2, 'Yes'),
 (4, '2입니다'),
 (288, '테스트 답변 '),
 (289, '아무거나 답변함\n업데이트 ㄹㄹㄹㄹ'),
 (305, 'ㅅㄷㄴㅅ1')]